Connect to elasticsearch

In [9]:
# Connection to Elasticsearch 
from pprint import pprint
from elasticsearch import Elasticsearch
from config.development import Config
 
es = Elasticsearch(
    Config.ES_HOST, 
    verify_certs=True, 
    basic_auth=(Config.ES_USERNAME, Config.ES_PASSWORD)
)

client_info = es.info()
print("Connected to the elasticsearch ✔")
pprint(client_info.body)

Connected to the elasticsearch ✔
{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'tdYmEtALQyuf5oSJGl74OQ',
 'name': 'david-server',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-06-19T01:37:57.847711500Z',
             'build_flavor': 'default',
             'build_hash': '580aff1a0064ce4c93293aaab6fcc55e22c10d1c',
             'build_snapshot': False,
             'build_type': 'deb',
             'lucene_version': '8.11.3',
             'minimum_index_compatibility_version': '6.0.0-beta1',
             'minimum_wire_compatibility_version': '6.8.0',
             'number': '7.17.29'}}


In [10]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [11]:
# make a dummy_data.json file to insert multiple documents in the index. We stored the id of each document in 
# document_ids list
import json 
from tqdm import tqdm 

document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index="my_index", body=document)
    document_ids.append(response['_id'])

100%|██████████| 5/5 [00:00<00:00, 31.04it/s]


In [12]:
document_ids

['KO498ZkBtfwvgmKSGMEm',
 'Ke498ZkBtfwvgmKSGMFj',
 'Ku498ZkBtfwvgmKSGMF_',
 'K-498ZkBtfwvgmKSGMGY',
 'LO498ZkBtfwvgmKSGMGv']

2. Delete Operation 
This is an example of successfull operation. If the ID exists in the document, delete operation wont return any error

In [13]:
response = es.delete(index="my_index", id=document_ids[0])

In [14]:
from pprint import pprint
pprint(response.body)

{'_id': 'KO498ZkBtfwvgmKSGMEm',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 5,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '_doc',
 '_version': 2,
 'result': 'deleted'}


In [15]:
try: 
    response = es.delete(index="my_index", id=2)
except Exception as e:
    print(e)

NotFoundError(404, "{'_index': 'my_index', '_type': '_doc', '_id': '2', '_version': 1, 'result': 'not_found', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 1}")
